In [1]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2hed
from skimage import exposure
from skimage import transform
from skimage import io
import tifffile as tiff
import matplotlib.image as mpimg
import os
os.add_dll_directory(r'C:\Users\77178\Documents\IBD-Biopsy-Slide-Imaging-Project\openslide-bin-4.0.0.5-windows-x64\bin')
import openslide as slide
import cv2
from skimage.util import img_as_ubyte
from tifffile import imwrite

In [3]:
def resize_image_to_target_size(input_path, output_path, target_size_kb):
    '''
    reszie the image to target_size_kb
    :param input_path: file path of input image
    :param output_path: where to save the resized image
    :param target_size_kb: to load in cellprofiler
    :return: image with target size
    '''
    # load image
    image = cv2.imread(input_path)
    
    # check if the image was loaded successfully
    if image is None:
        raise ValueError(f"Failed to load image from {input_path}. Please check the file path or format.")
    
    file_size_kb = os.path.getsize(input_path) / 1024  # initial file size in KB

    # if already under the target size, save it directly
    if file_size_kb <= target_size_kb:
        print(f"No resizing needed for {input_path}, file size is already under {target_size_kb} KB.")
        imwrite(output_path, image)
        return image

    # scale factor
    scale_factor = 0.95

    # iterate to resize until file size is under the target size
    while file_size_kb > target_size_kb:
        # calculate new dimensions
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        # resize the image
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
        # save temporarily to check file size
        temp_output_path = "temp_resized_image.png"
        cv2.imwrite(temp_output_path, resized_image)
        file_size_kb = os.path.getsize(temp_output_path) / 1024
        # reduce scale factor if still above limit
        scale_factor *= 0.95

    # load resized image back and save
    resized_image = cv2.imread(temp_output_path)
    os.remove(temp_output_path)  # clean up temporary file
    imwrite(output_path, resized_image)  # save
    print(f"Resized image saved as {output_path}")
    return resized_image

In [4]:
def convert_to_multichannel(input_path, output_path):
    '''
    extract the hematoxylin and eosin stain form images to create a multichannel image
    :param input_path: path of input image
    :param output_path: path of output image
    :return: multichannel image with h&e channel
    '''
    # load images
    resized_image = cv2.imread(input_path)
    
    # convert to HED and separate channels
    image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)  # convert to RGB
    image_hed = rgb2hed(image_rgb)  # Convert RGB to HED

    # normalize H&E channels to [0, 1] range
    hematoxylin = (image_hed[:, :, 0] - image_hed[:, :, 0].min()) / (image_hed[:, :, 0].max() - image_hed[:, :, 0].min())
    eosin = (image_hed[:, :, 1] - image_hed[:, :, 1].min()) / (image_hed[:, :, 1].max() - image_hed[:, :, 1].min())

    # create an empty channel
    empty_channel = np.zeros_like(hematoxylin)
    # combine channels
    multichannel_image = np.stack((hematoxylin, eosin, empty_channel), axis=-1)

    # save the final multichannel image
    imwrite(output_path, multichannel_image)
    print(f"Multichannel image saved as {output_path}")

In [5]:
# paths
input_dir = r"C:/Users/77178/Documents/IBD-Biopsy-Slide-Imaging-Project-data/combined_data/"
resized_output_dir = r"C:/Users/77178/Documents/IBD-Biopsy-Slide-Imaging-Project-data/temp_resize/resize/"
multichannel_output_dir = r"C:/Users/77178/Documents/IBD-Biopsy-Slide-Imaging-Project-data/multichannel_resized/"


In [6]:
# target size in kb
target_size_kb = 34000

In [7]:
# process all images in the input directory
for filename in os.listdir(input_dir):
    # ignore hidden macOS files that start with '._'
    if filename.startswith("._"):
        continue

    # proceed only if it ends with .tif or .tiff
    if filename.endswith(".tif") or filename.endswith(".tiff"):
        input_path = os.path.join(input_dir, filename)
        resized_output_path = os.path.join(resized_output_dir, f"{os.path.splitext(filename)[0]}_resized.tif")
        print(f"Processing file: {filename}")
        resize_image_to_target_size(input_path, resized_output_path, target_size_kb)


Processing file: S24-888 D.ome.tif
Resized image saved as C:/Users/77178/Documents/IBD-Biopsy-Slide-Imaging-Project-data/temp_resize/resize/S24-888 D.ome_resized.tif


In [8]:
# convert resized images to multichannel and save
for filename in os.listdir(resized_output_dir):
    if filename.endswith("_resized.tif"):
        resized_input_path = os.path.join(resized_output_dir, filename)
        multichannel_output_path = os.path.join(multichannel_output_dir, f"{os.path.splitext(filename)[0]}_multichannel.tif")
        print(f"Converting to multichannel: {filename}")
        convert_to_multichannel(resized_input_path, multichannel_output_path)

print("Processing complete for all files.")

Converting to multichannel: S24-888 D.ome_resized.tif
Multichannel image saved as C:/Users/77178/Documents/IBD-Biopsy-Slide-Imaging-Project-data/temp_resize/multi/S24-888 D.ome_resized_multichannel.tif
Processing complete for all files.
